In [ ]:
CONFIG = {
    'full_dataset' : "../data/full_dataset.csv",
    'target' : 'Mean_freq',
    'SEQ_LEN' : 400,
    'model_name' : 'TCN'
    }
features = {"Rainfall": True,"Temp" : True, "Wind" : True, "Pressure" : True, "Humidity" : True}
CONFIG['features'] = [k for k,v in features.items() if v]

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tcn import TCN
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error
import random
import os

In [2]:
def set_seed(seed):
    # Python random module
    random.seed(seed)
    
    # Numpy random module
    np.random.seed(seed)
    
    # PyTorch random seed
    torch.manual_seed(seed)
    
    # Ensures deterministic behavior for GPU
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

    #Change it if you want deterministic behaviour
    torch.backends.cudnn.benchmark = True
    torch.use_deterministic_algorithms(False)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"
    

# Set your desired seed
SEED = 42
set_seed(SEED)

In [ ]:
train_rmses = []
val_rmses = []
test_rmses = []
model_name = CONFIG['model_name']
best_val_rmse = float('inf')
best_test_rmse = float('inf')


for i in range (10):
    print(f"Iteration {i}")
    # Load data
    df = pd.read_csv(CONFIG['full_dataset'], sep=",")

    #TODO: REMOVE THIS
    df = df.iloc[400:]

    df.drop(columns=['Mean_am', 'Std_am', 'Unnamed: 0'], axis=1, inplace=True)

    # Define target and features
    target = CONFIG['target']

    split_date_earthquake = '2024-08-13'

    normal_df = df[df['Dates UTC'] < split_date_earthquake].copy()
    test_df = df[df['Dates UTC'] >= split_date_earthquake].copy()

    split_index = len(normal_df)
    # Split data
    X_train = normal_df[CONFIG['features']]
    y_train = normal_df[target]
    X_test = test_df[CONFIG['features']]
    y_test = test_df[target]

    # Scale data separately to avoid data leakage
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

    X_test_scaled = scaler_X.transform(X_test)
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

    X = np.concat([X_train_scaled, X_test_scaled])
    y = np.concat([y_train_scaled, y_test_scaled])

    SEQ_LEN = CONFIG['SEQ_LEN']

    # Convert data into sequences
    def create_sequences(X, y, seq_len):
        X_seq, y_seq = [], []
        for i in range(len(X) - seq_len):
            X_seq.append(X[i:i+seq_len])  
            y_seq.append(y[i+seq_len])    
        return np.array(X_seq), np.array(y_seq)

    X_seq, y_seq = create_sequences(X, y, SEQ_LEN)

    #Importante per azzeccare il punto giusto del terremoto
    split_index = split_index - SEQ_LEN
    X_seq_train = X_seq[:split_index]
    X_test = X_seq[split_index:]
    y_seq_train = y_seq[:split_index]
    y_test = y_seq[split_index:]


    X_train, X_val, y_train, y_val = train_test_split(X_seq_train, y_seq_train, test_size=0.1, random_state=42, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).transpose(1, 2).to(device)  # Shape: (N, C, L)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).transpose(1, 2).to(device)  # Shape: (N, C, L)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).transpose(1, 2).to(device)  # Shape: (N, C, L)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)  # Shape: (N, 1)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device)  # Shape: (N, 1)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)  # Shape: (N, 1)

    def seed_worker(worker_id):
        np.random.seed(worker_id)
        random.seed(worker_id)
        torch.manual_seed(worker_id)

    seed = 42
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # Create PyTorch datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(seed))
    val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
    item = next(iter(train_loader))
    # Define TCN Model
    class TCNRegression(nn.Module):
        def __init__(self, input_dim, output_dim):
            super(TCNRegression, self).__init__()
            self.tcn = TCN(
                num_inputs=input_dim,
                num_channels=[16, 16, 16, 16, 16, 16, 8],
                kernel_size=6,
                dropout=0.3,
                causal=True,
                use_skip_connections=True,
            )
            self.linear = nn.Linear(8, 4)
            self.linear_2 = nn.Linear(4, output_dim)

        def forward(self, x):
            x = self.tcn(x)  # Output shape: (N, C, L)
            x = x[:, :, -1]  # Take only the last time step
            x = self.linear(x)
            x = self.linear_2(x)
            return x

    # Initialize model
    model = TCNRegression(input_dim=5, output_dim=1)

    train_model = True
    if train_model:
        model.to(device)
        # Define loss and optimizer
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        best_val_loss = float('inf')  # Initialize best validation loss

        for epoch in range(25):
            model.train()
            train_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                output = model(X_batch).squeeze()
                loss = criterion(output, y_batch.squeeze())
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss /= len(train_loader)  # Compute average train loss

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    output = model(X_batch).squeeze()
                    val_loss += criterion(output, y_batch.squeeze()).item()

            val_loss /= len(val_loader)  # Compute average validation loss

            # Save the model if it has the best validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), model_name + ".pth")
                print(f"Epoch {epoch+1}: Best model saved with Val Loss = {val_loss:.4f}")

            print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    if train_model:
        device = "cpu"
        model.load_state_dict(torch.load(model_name + ".pth", weights_only=True))
        model.to(device)
        model.eval()
        # Move tensors to CPU
        X_train_tensor = X_train_tensor.to(device)
        y_train_tensor = y_train_tensor.to(device)
        X_val_tensor = X_val_tensor.to(device)
        y_val_tensor = y_val_tensor.to(device)
        X_test_tensor = X_test_tensor.to(device)
        y_test_tensor = y_test_tensor.to(device)

        # Predictions
        with torch.no_grad():
            train_predictions = model(X_train_tensor).detach().cpu().numpy()
            val_predictions = model(X_val_tensor).detach().cpu().numpy()
            test_predictions = model(X_test_tensor).detach().cpu().numpy()

        # Move actual values to CPU
        y_train_actual = y_train_tensor.view(-1, 1).cpu().numpy()
        y_val_actual = y_val_tensor.view(-1, 1).cpu().numpy()
        y_test_actual = y_test_tensor.view(-1, 1).cpu().numpy()

        # Scale back using scaler_y
        train_predictions = scaler_y.inverse_transform(train_predictions)
        val_predictions = scaler_y.inverse_transform(val_predictions)
        test_predictions = scaler_y.inverse_transform(test_predictions)

        y_train_actual = scaler_y.inverse_transform(y_train_actual)
        y_val_actual = scaler_y.inverse_transform(y_val_actual)
        y_test_actual = scaler_y.inverse_transform(y_test_actual)

        df_cut = df.tail(len(df) - SEQ_LEN).copy()
        # Ensure 'Dates UTC' is a datetime object
        df_cut['Dates UTC'] = pd.to_datetime(df_cut['Dates UTC'])
        normal_df = df_cut[df_cut['Dates UTC'] < split_date_earthquake].copy()
        test_df = df_cut[df_cut['Dates UTC'] >= split_date_earthquake].copy()

        # Define conservative recovery period
        recovery_start = pd.to_datetime('2024-08-13')
        recovery_end = recovery_start + pd.DateOffset(days=2.5*30)

        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predictions))
        val_rmse = np.sqrt(mean_squared_error(y_val_actual, val_predictions))
        #test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predictions))
        # Filter test data for the recovery period
        recovery_test_indices = test_df['Dates UTC'] >= recovery_end
        recovery_test_actual = y_test_actual[recovery_test_indices]
        recovery_test_predictions = test_predictions[recovery_test_indices]

        # Calculate RMSE for the recovery period
        test_rmse = np.sqrt(mean_squared_error(recovery_test_actual, recovery_test_predictions))
        print(f"Train RMSE: {train_rmse:.4f}, Val RMSE {val_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
        # Exclude recovery period from the test data for RMSE calculation
        non_recovery_test_indices = test_df['Dates UTC'] < recovery_start
        non_recovery_test_actual = y_test_actual[non_recovery_test_indices]
        non_recovery_test_predictions = test_predictions[non_recovery_test_indices]

        # Calculate RMSE for the whole dataset excluding the recovery period
        overall_rmse = np.sqrt(mean_squared_error(np.concatenate((y_train_actual, y_val_actual, non_recovery_test_actual)),
                                                np.concatenate((train_predictions, val_predictions, non_recovery_test_predictions))))
        if test_rmse < best_test_rmse:
            best_test_rmse = test_rmse
            torch.save(model.state_dict(), f'../models/{model_name}_final.pth')
            print(f"Best FINAL model saved with test rmse = {test_rmse:.4f}")

        train_rmses.append(train_rmse)
        val_rmses.append(val_rmse)
        test_rmses.append(test_rmse)

SyntaxError: invalid syntax (3821982259.py, line 14)

In [ ]:
# Compute mean and standard error of the mean (stderr) for the RMSE lists
train_rmse_mean = np.mean(train_rmses)
train_rmse_stderr = np.std(train_rmses) / np.sqrt(len(train_rmses))

val_rmse_mean = np.mean(val_rmses)
val_rmse_stderr = np.std(val_rmses) / np.sqrt(len(val_rmses))

test_rmse_mean = np.mean(test_rmses)
test_rmse_stderr = np.std(test_rmses) / np.sqrt(len(test_rmses))

print(f"Train RMSE Mean: {train_rmse_mean:.4f}, Train RMSE StdErr: {train_rmse_stderr:.4f}, Train RMSE StdDev: {np.std(train_rmses):.4f}")
print(f"Val RMSE Mean: {val_rmse_mean:.4f}, Val RMSE StdErr: {val_rmse_stderr:.4f}, Val RMSE StdDev: {np.std(val_rmses):.4f}")
print(f"Test RMSE Mean: {test_rmse_mean:.4f}, Test RMSE StdErr: {test_rmse_stderr:.4f}, Test RMSE StdDev: {np.std(test_rmses):.4f}")

Train RMSE Mean: 0.0229, Train RMSE StdErr: 0.0002, Train RMSE StdDev: 0.0006
Val RMSE Mean: 0.0303, Val RMSE StdErr: 0.0000, Val RMSE StdDev: 0.0001
Test RMSE Mean: 0.0353, Test RMSE StdErr: 0.0001, Test RMSE StdDev: 0.0002


In [ ]:
device = "cpu"
model.load_state_dict(torch.load(f'../models/{model_name}_final.pth', weights_only=False))
model.to(device)
model.eval()
# Move tensors to GPU
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Predictions
with torch.no_grad():
    train_predictions = model(X_train_tensor).detach().cpu().numpy()
    val_predictions = model(X_val_tensor).detach().cpu().numpy()
    test_predictions = model(X_test_tensor).detach().cpu().numpy()

# Move actual values to CPU
y_train_actual = y_train_tensor.view(-1, 1).cpu().numpy()
y_val_actual = y_val_tensor.view(-1, 1).cpu().numpy()
y_test_actual = y_test_tensor.view(-1, 1).cpu().numpy()

# Scale back using scaler_y
train_predictions = scaler_y.inverse_transform(train_predictions)
val_predictions = scaler_y.inverse_transform(val_predictions)
test_predictions = scaler_y.inverse_transform(test_predictions)

y_train_actual = scaler_y.inverse_transform(y_train_actual)
y_val_actual = scaler_y.inverse_transform(y_val_actual)
y_test_actual = scaler_y.inverse_transform(y_test_actual)

In [ ]:
df_cut = df.tail(len(df) - SEQ_LEN).copy()
# Ensure 'Dates UTC' is a datetime object
df_cut['Dates UTC'] = pd.to_datetime(df_cut['Dates UTC'])
normal_df = df_cut[df_cut['Dates UTC'] < split_date_earthquake].copy()
test_df = df_cut[df_cut['Dates UTC'] >= split_date_earthquake].copy()

# Define conservative recovery period
recovery_start = pd.to_datetime('2024-08-13')
recovery_end = recovery_start + pd.DateOffset(days=2.5*30)

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predictions))
val_rmse = np.sqrt(mean_squared_error(y_val_actual, val_predictions))
#test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predictions))
# Filter test data for the recovery period
recovery_test_indices = test_df['Dates UTC'] >= recovery_end
recovery_test_actual = y_test_actual[recovery_test_indices]
recovery_test_predictions = test_predictions[recovery_test_indices]

# Calculate RMSE for the recovery period
test_rmse = np.sqrt(mean_squared_error(recovery_test_actual, recovery_test_predictions))
print(f"Train RMSE: {train_rmse:.4f}, Val RMSE {val_rmse:.4f}, Test RMSE: {test_rmse:.4f}")
# Exclude recovery period from the test data for RMSE calculation
non_recovery_test_indices = test_df['Dates UTC'] < recovery_start
non_recovery_test_actual = y_test_actual[non_recovery_test_indices]
non_recovery_test_predictions = test_predictions[non_recovery_test_indices]

# Calculate RMSE for the whole dataset excluding the recovery period
overall_rmse = np.sqrt(mean_squared_error(np.concatenate((y_train_actual, y_val_actual, non_recovery_test_actual)),
                                          np.concatenate((train_predictions, val_predictions, non_recovery_test_predictions))))
print(f"Overall RMSE (excluding recovery period): {overall_rmse:.4f}")

# Calculate residuals
train_residuals = np.abs(y_train_actual.flatten() - train_predictions.flatten())
val_residuals = np.abs(y_val_actual.flatten() - val_predictions.flatten())
test_residuals = np.abs(y_test_actual.flatten() - test_predictions.flatten())

# Moving averages
ma_window = 100
train_residuals_ma = np.convolve(train_residuals, np.ones(ma_window)/ma_window, mode='valid')
val_residuals_ma = np.convolve(val_residuals, np.ones(ma_window)/ma_window, mode='valid')
test_residuals_ma = np.convolve(test_residuals, np.ones(ma_window)/ma_window, mode='valid')

train_indices = normal_df['Dates UTC'][:len(train_predictions)]
val_indices = normal_df['Dates UTC'][len(train_predictions):(len(train_predictions) + len(val_predictions))]
test_indices = test_df['Dates UTC'][:len(test_predictions)]

# Create subplots with shared x-axis
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
)

# Plot actual vs predicted
fig.add_trace(go.Scatter(x=train_indices, y=y_train_actual.flatten(), mode='lines', line=dict(color='blue'), name='Actual', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=train_indices, y=train_predictions.flatten(), mode='lines', line=dict(color='red'), name='Train Predicted', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=val_indices, y=y_val_actual.flatten(), mode='lines', line=dict(color='blue'), name='Validation Actual', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=val_indices, y=val_predictions.flatten(), mode='lines', line=dict(color='orange'), name='Validation Predicted', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=test_indices, y=y_test_actual.flatten(), mode='lines', line=dict(color='blue'), name='Test Actual', showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=test_indices, y=test_predictions.flatten(), mode='lines', line=dict(color='green'), name='Test Predicted', showlegend=False), row=1, col=1)

# Highlight recovery period
fig.add_vrect(x0=recovery_start, x1=recovery_end, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0, row=1, col=1)
fig.add_vrect(x0=recovery_start, x1=recovery_end, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0, row=2, col=1)

# Plot residuals as lines
fig.add_trace(go.Scatter(x=train_indices, y=train_residuals, mode='lines', line=dict(color='red'), name='Train Residuals', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=val_indices, y=val_residuals, mode='lines', line=dict(color='orange'), name='Validation Residuals', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=test_indices, y=test_residuals, mode='lines', line=dict(color='green'),name='Test Residuals', showlegend=False), row=2, col=1)

# Plot moving averages of residuals
fig.add_trace(go.Scatter(x=train_indices[ma_window-1:], y=train_residuals_ma, mode='lines', line=dict(color='black'), name='Train Residuals <br> Moving Average', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=val_indices[ma_window-1:], y=val_residuals_ma, mode='lines', line=dict(color='black'), name='Validation Residuals MA', showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=test_indices[ma_window-1:], y=test_residuals_ma, mode='lines', line=dict(color='black'), name='Test Residuals MA', showlegend=False), row=2, col=1)

OFFSET = 200
x_start_offset = normal_df['Dates UTC'].iloc[OFFSET]  # Start x-axis at the 100th sample
x_end = test_df['Dates UTC'].iloc[-1]  # Keep the full range

fig.update_layout(
    xaxis2={  # X-axis for the second subplot
        "title": "Date",
        "titlefont": {"size": 22},  
        "tickfont": {"size": 16},
        "range": [x_start_offset, x_end]  # Apply visual offset``  
    },
    yaxis={
        "title": "Frequency (Hz)",
        "titlefont": {"size": 22},  # Increase Y-axis label font size (for the first subplot)
        "tickfont": {"size": 16}  
    },
    yaxis2={  # Y-axis for the second subplot
        "title": "Residuals",
        "titlefont": {"size": 22},  
        "tickfont": {"size": 16}  
    },
    legend={
        "font": {"size": 24}  # Increase legend font size
    },
    height=600
)

# Show the figure
fig.show()
fig.write_image("../images/TCN_no_legend.svg", width=1800, height=600)


Train RMSE: 0.0225, Val RMSE 0.0302, Test RMSE: 0.0350
Overall RMSE (excluding recovery period): 0.0233


In [ ]:
save_last_layer = False
if save_last_layer:
    # Load the best model
    model.load_state_dict(torch.load(f'../models/{model_name}_final.pth', weights_only=False))
    model.to(device)
    model.eval()
    # Get the outputs of the last layer over the whole dataset
    with torch.no_grad():
        train_outputs = model.linear(model.tcn(X_train_tensor)[:, :, -1]).detach().cpu().numpy()
        val_outputs = model.linear(model.tcn(X_val_tensor)[:, :, -1]).detach().cpu().numpy()
        test_outputs = model.linear(model.tcn(X_test_tensor)[:, :, -1]).detach().cpu().numpy()

    # Combine all outputs into a single array
    all_outputs = np.concatenate((train_outputs, val_outputs, test_outputs))

    # Combine all dates into a single array
    all_dates = np.concatenate((train_indices, val_indices, test_indices))

    # Combine all scaled targets into a single array
    all_scaled_targets = np.concatenate((y_train_tensor.cpu().numpy(), y_val_tensor.cpu().numpy(), y_test_tensor.cpu().numpy()))

    # Create a dataframe with the dates, outputs, and scaled targets
    outputs_df = pd.DataFrame(all_outputs, columns=[f"feature_{i}" for i in range(all_outputs.shape[1])])
    outputs_df['scaled_target'] = all_scaled_targets.flatten()

    # Save the dataframe to a CSV file
    outputs_df.to_csv("../transformed_data/TCN_last_layer.csv", index=False)
